#### To understand optimize gradient descent, Checking gradients using gradient Checking
Analytical processer and takes longer time to obtained approximate gradient 

In [3]:
## importing python libraries
import numpy as np

In [4]:

def activation_function(Z, typ):
    if typ == 'sig':
        A = 1/(1+np.exp(-Z))
        assert A.shape == Z.shape
        return A
    elif typ == 'tanh':
        A = np.tanh(Z)
        assert A.shape == Z.shape
        return A
    elif typ == 'relu':
        A = np.maximum(0, Z)
        assert A.shape == Z.shape
        return A


def dictionary_to_vector(parameters):
    """
    Roll all our parameters dictionary into a single vector satisfying our specific required shape.
    """
    para_list=parameters.keys()
    keys = []
    count = 0
    for key in para_list:
        # flatten parameter
        new_vector = np.reshape(parameters[key], (-1, 1))
        keys = keys + [key] * new_vector.shape[0]
        
        if count == 0:
            theta = new_vector
        else:
            theta = np.concatenate((theta, new_vector), axis=0)
        count = count + 1

    return theta, keys

def vector_to_dictionary(theta):
    """
    Unroll all our parameters dictionary from a single vector satisfying our specific required shape.
    """
    parameters = {}
    parameters["w1"] = theta[: 20].reshape((5, 4))
    parameters["b1"] = theta[20: 25].reshape((5, 1))
    parameters["w2"] = theta[25: 40].reshape((3, 5))
    parameters["b2"] = theta[40: 43].reshape((3, 1))
    parameters["w3"] = theta[43: 46].reshape((1, 3))
    parameters["b3"] = theta[46: 47].reshape((1, 1))

    return parameters



def gradients_to_vector(gradients):
    """
    Roll all our gradients dictionary into a single vector satisfying our specific required shape.
    """
    grad_list=gradients.keys()
    start_letter1='w'
    start_letter2='b'
    grad_list=[x for x in grad_list if (x.startswith(start_letter1))]
    print(grad_list)
    count = 0
    for key in ["dw1", "db1", "dw2", "db2", "dw3", "db3"]:
        # flatten parameter
        new_vector = np.reshape(gradients[key], (-1, 1))
        
        if count == 0:
            theta = new_vector
        else:
            theta = np.concatenate((theta, new_vector), axis=0)
        count = count + 1

    return theta

### Running test cases to check if the NN is working or not

In [5]:
!pytest test_NN_deep_layer.py 

============================= test session starts =============================
platform win32 -- Python 3.7.4, pytest-5.4.2, py-1.8.1, pluggy-0.13.1
rootdir: C:\Users\VISWAMBHAR YASA\ML\Deep_learning\DEEP_LEARNING\Neural_network\Deep_layered_NN
collected 4 items

test_NN_deep_layer.py ....                                               [100%]

============================== 4 passed in 0.16s ==============================


## Multi layer Neural networt

In [6]:
def forward_propagation_n(X, Y, parameters):
    m = X.shape[1]
    W1 = parameters['w1']
    b1 = parameters['b1']
    W2 = parameters['w2']
    b2 = parameters['b2']
    W3 = parameters['w3']
    b3 = parameters['b3']

    # LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SIGMOID
    Z1 = np.dot(W1, X) + b1
    A1 = activation_function(Z1, 'relu')
    Z2 = np.dot(W2, A1) + b2
    A2 = activation_function(Z2, 'relu')
    Z3 = np.dot(W3, A2) + b3
    A3 = activation_function(Z3, 'sig')

    # Cost
    log_probs = np.multiply(-np.log(A3),Y) + np.multiply(-np.log(1 - A3), 1 - Y)
    cost = 1. / m * np.sum(log_probs)
    
    cache = (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3)
    
    return cost, cache

def backward_propagation_n(X, Y, cache):
    m = X.shape[1]
    (Z1, A1, W1, b1, Z2, A2, W2, b2, Z3, A3, W3, b3) = cache
    
    dZ3 = A3 - Y
    dW3 = 1. / m * np.dot(dZ3, A2.T)
    db3 = 1. / m * np.sum(dZ3, axis=1, keepdims=True)
    
    dA2 = np.dot(W3.T, dZ3)
    dZ2 = np.multiply(dA2, np.int64(A2 > 0))
    dW2 = 1. / m * np.dot(dZ2, A1.T) * 2
    db2 = 1. / m * np.sum(dZ2, axis=1, keepdims=True)
    
    dA1 = np.dot(W2.T, dZ2)
    dZ1 = np.multiply(dA1, np.int64(A1 > 0))
    dW1 = 1. / m * np.dot(dZ1, X.T)
    db1 = 4. / m * np.sum(dZ1, axis=1, keepdims=True)
    
    gradients = {"dZ3": dZ3, "dw3": dW3, "db3": db3,
                 "dA2": dA2, "dZ2": dZ2, "dw2": dW2, "db2": db2,
                 "dA1": dA1, "dZ1": dZ1, "dw1": dW1, "db1": db1}
    
    return gradients

In [7]:

def gradient_check(parameters, gradients, X, Y, epsilon=1e-7, print_msg=True):
    """
    Checks if backward_propagation_n computes correctly the gradient of the cost output by forward_propagation_n
    """

    parameters_values, _ = dictionary_to_vector(parameters)
    
    grad = gradients_to_vector(gradients)
    num_parameters = parameters_values.shape[0]
    J_plus = np.zeros((num_parameters, 1))
    J_minus = np.zeros((num_parameters, 1))
    gradapprox = np.zeros((num_parameters, 1))
    
    for i in range(num_parameters):
        theta_plus=np.copy(parameters_values)
        theta_plus[i][0]=theta_plus[i][0]+epsilon
        J_plus[i], cache_plus=forward_propagation_n(X,Y,vector_to_dictionary(theta_plus))
        

        theta_minus=np.copy(parameters_values)
        theta_minus[i][0]=theta_minus[i][0]-epsilon
        J_minus[i], cache_minus=forward_propagation_n(X,Y,vector_to_dictionary(theta_minus))
        
        gradapprox[i] = (J_plus[i] - J_minus[i])/(2 * epsilon)
        

    numerator = np.linalg.norm(grad - gradapprox)                                           
    denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)                                        
    difference = numerator/(denominator*1.0)
    
    if print_msg:
        if difference > 2e-7:
            print ("\033[93m" + "There is a mistake in the backward propagation! difference = " + str(difference) + "\033[0m")
        else:
            print ("\033[92m" + "Your backward propagation works perfectly fine! difference = " + str(difference) + "\033[0m")

    return difference

In [8]:
np.random.seed(1)
x = np.random.randn(4,3)
y = np.array([1, 1, 0])
w1 = np.random.randn(5,4) 
b1 = np.random.randn(5,1) 
w2 = np.random.randn(3,5) 
b2 = np.random.randn(3,1) 
w3 = np.random.randn(1,3) 
b3 = np.random.randn(1,1) 
parameters = {'w1': w1,
                  'b1': b1,
                  'w2': w2,
                  'b2': b2,
                  'w3': w3,
                  'b3': b3}

In [9]:
AL, fw_cache=forward_propagation_n(x,y, parameters)
gradient=backward_propagation_n(x, y, fw_cache)
diff=gradient_check(parameters, gradient, x, y, epsilon=1e-7, print_msg=True)

[]
There is a mistake in the backward propagation! difference = 0.28509315677616237


### Using gradient check, gradients obtained are accurate or not